# Segmenting and Clustering Neighborhoods in Toronto

### Import Libraries

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

### Scrape the List of postal codes of Canada Wikipedia page and create a DataFrame

In [2]:
# Download url data from Wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
soup = BeautifulSoup(source, 'lxml')
soup

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XoFTzgpAICoAAD@fNTsAAABO","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":948084252,"wgRevisionId":948084252,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"

In [3]:
# Extract table from the HTML doc
table = soup.find_all('table')[0]
html_table = pd.read_html(str(table))
html_table

[    Postal code           Borough  \
 0           M1A      Not assigned   
 1           M2A      Not assigned   
 2           M3A        North York   
 3           M4A        North York   
 4           M5A  Downtown Toronto   
 5           M6A        North York   
 6           M7A  Downtown Toronto   
 7           M8A      Not assigned   
 8           M9A         Etobicoke   
 9           M1B       Scarborough   
 10          M2B      Not assigned   
 11          M3B        North York   
 12          M4B         East York   
 13          M5B  Downtown Toronto   
 14          M6B        North York   
 15          M7B      Not assigned   
 16          M8B      Not assigned   
 17          M9B         Etobicoke   
 18          M1C       Scarborough   
 19          M2C      Not assigned   
 20          M3C        North York   
 21          M4C         East York   
 22          M5C  Downtown Toronto   
 23          M6C              York   
 24          M7C      Not assigned   
 25         

In [4]:
# # Create a DataFrame from html_table
df=pd.DataFrame(html_table[0])
print(df.dtypes)
df.head(10)

Postal code     object
Borough         object
Neighborhood    object
dtype: object


,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
7,M8A,Not assigned,NaN
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge


### Clean DataFrame

In [5]:
# Change column names per assignment requirements
df.rename(columns={'Postal code': 'PostalCode'}, inplace=True)
df.head(10)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
7,M8A,Not assigned,NaN
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge


In [6]:
#  Drop rows with Borough = Not assigned
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
df.head(10)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge
11,M3B,North York,Don Mills
12,M4B,East York,Parkview Hill / Woodbine Gardens
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [7]:
# More than one neighborhood can exist in one postal code area
# Combining neighbourhoods with the same PostalCode
Toronto_df = df.groupby(['PostalCode','Borough'], sort=False).agg(', '.join)

# resetting index
Toronto_df.reset_index(inplace=True)
Toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [8]:
# Replace ' /' with ','
Toronto_df.Neighborhood = Toronto_df.Neighborhood.str.replace(r" /",',')
Toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [9]:
# replacing the name of the blank and 'Not assigned' neighborhoods with the name of the borough
Toronto_df['Neighborhood'] = np.where(Toronto_df['Neighborhood'] == '',Toronto_df['Borough'], Toronto_df['Neighborhood'])
Toronto_df['Neighborhood'] = np.where(Toronto_df['Neighborhood'] == 'Not assigned',Toronto_df['Borough'], Toronto_df['Neighborhood'])

Toronto_df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [10]:
Toronto_df.shape

(103, 3)

# Adding Coordinates

### Retrieve coordinates data

In [11]:
# Access csv file with geographical coordinates for each postal code
coordinates = pd.read_csv('https://cocl.us/Geospatial_data')
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
# Change column names as per assignment 
coordinates.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Merge DataFrames

In [13]:
Toronto_coords = pd.merge(Toronto_df,coordinates,on='PostalCode')
Toronto_coords.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# Explore and cluster the neighborhoods in Toronto

In [19]:
# Convert an address into latitude and longitude values
!pip install geopy
from geopy.geocoders import Nominatim 
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [20]:
!pip install folium
import folium # map rendering library

### Create a map of Toronto

In [21]:
# Create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# Add markers to map
for lat, lng, label in zip(Toronto_coords['Latitude'], Toronto_coords['Longitude'], Toronto_coords['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Cluster Neighborhoods

In [22]:
# Use k-means to cluster the neighborhood into 5 clusters based on geographical situation. We will split our data into 5 clusters
from sklearn.cluster import KMeans 
kclusters = 5

toronto_cluster = Toronto_coords.drop(['PostalCode','Borough','Neighborhood'],1)

# Run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_cluster)

# Check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 4, 2, 3, 2, 1, 0, 4, 4, 2], dtype=int32)

In [23]:
# Add Cluster labels
Toronto_coords.insert(0, 'Cluster Labels', kmeans.labels_)
Toronto_coords.head()

,Cluster Labels,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,4,M3A,North York,Parkwoods,43.753259,-79.329656
1,4,M4A,North York,Victoria Village,43.725882,-79.315572
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,2,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [24]:
Toronto_coords.shape

(103, 6)

In [25]:
# Import Matplotlib and associated plotting modules
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

# Create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# Set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_coords['Latitude'], Toronto_coords['Longitude'], Toronto_coords['Neighborhood'], Toronto_coords['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Observations:
Looking at map_clusters, we can clearly see the diferentiation ofr the five clusters so our choice for k value of 5 has been correct.

# Analyse Neighborhoods

### Extract venues data for each neighborhood from Foursquare

In [31]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: WMPBKRQUACQ55PNH425NOML3QWJOBTEBNW4TGDFY2SGGXYDO
CLIENT_SECRET:34OQEWXO22WVN4U3JJVQ5EKTIYTVRSXSU1KL0TB50WDH4DVA


In [28]:
radius=500
LIMIT=100

In [39]:
# Get json file data from Foursquare for venues in Toronto neighborhoods
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e81cb06aba297001b581276'},
 'response': {'headerLocation': 'Islington - City Centre West',
  'headerFullLocation': 'Islington - City Centre West, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 13,
  'suggestedBounds': {'ne': {'lat': 43.6333408045, 'lng': -79.51479402615583},
   'sw': {'lat': 43.6243407955, 'lng': -79.52720477384419}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bc9f9b6b6c49c7469688f91',
       'name': 'South St. Burger',
       'location': {'address': '1020 Islington Ave',
        'lat': 43.63131374571208,
        'lng': -79.51840794740386,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.63131374571208,
          'lng': -79.51840794740386}],
        'distance': 345

In [40]:
# Set up a function to extract Toronto nearby venues
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [41]:
# Extract venues nearby Toronto
toronto_venues = getNearbyVenues(names=Toronto_coords['Neighborhood'],
                                   latitudes=Toronto_coords['Latitude'],
                                   longitudes=Toronto_coords['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview
The Danforth West, Ri

In [42]:
toronto_venues.head(5)

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.753259,-79.329656,Bella Vita Catering & Private Chef Service,43.756651,-79.331524,BBQ Joint
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [43]:
toronto_venues.shape

(2185, 7)

In [44]:
# See venues for each neighborhood
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Wilson Heights, Downsview North",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
Berczy Park,55,55,55,55,55,55
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
Business reply mail Processing CentrE,18,18,18,18,18,18


### One Hot Encoding 

In [46]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# Add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# Move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot.head()

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Neighbourhood
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Parkwoods
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Parkwoods
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Parkwoods
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Victoria Village
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Victoria Village


In [48]:
# Group onehot results by Neighborhood
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Alderwood, Long Branch",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Bayview Village,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"Bedford Park, Lawrence Manor East",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,"Birch Cliff, Cliffside West",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Brockton, Parkdale Village, Exhibition Place",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667
8,Business reply mail Processing CentrE,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556
9,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.000000,0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [53]:
# Calculate frequency of venues based on their number per Neighborhood; display top 5 per neighborhood
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Clothing Store  0.25
1                     Lounge  0.25
2             Breakfast Spot  0.25
3  Latin American Restaurant  0.25
4         Mexican Restaurant  0.00


----Alderwood, Long Branch----
                venue  freq
0         Pizza Place  0.22
1            Pharmacy  0.11
2      Sandwich Place  0.11
3                 Pub  0.11
4  Athletics & Sports  0.11


----Bathurst Manor, Wilson Heights, Downsview North----
           venue  freq
0           Bank  0.11
1    Coffee Shop  0.11
2    Bridal Shop  0.06
3  Deli / Bodega  0.06
4     Restaurant  0.06


----Bayview Village----
                 venue  freq
0                 Café  0.25
1                 Bank  0.25
2  Japanese Restaurant  0.25
3   Chinese Restaurant  0.25
4        Movie Theater  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0     Thai Restaurant  0.08
1         Coffee Shop  0.08
2          Restaurant  0.08
3      Sandwich Pla

In [54]:
# Sort in Descending order to bring top ones on top
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [55]:
# Create a DataFrame showing ranking venues categories based on frequency per neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Breakfast Spot,Clothing Store,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
1,"Alderwood, Long Branch",Pizza Place,Gym,Athletics & Sports,Pharmacy,Pub,Sandwich Place,Skating Rink,Coffee Shop,Airport Terminal,Deli / Bodega
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Middle Eastern Restaurant,Fried Chicken Joint,Sandwich Place,Bridal Shop,Diner,Restaurant,Deli / Bodega,Supermarket
3,Bayview Village,Café,Japanese Restaurant,Bank,Chinese Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Dessert Shop
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Restaurant,Italian Restaurant,Thai Restaurant,Indian Restaurant,Pub,Butcher,Sushi Restaurant,Liquor Store


In [112]:
toronto_grouped.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.04,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Cluster Neighborhoods

In [65]:
# Group neighbourhoods and venues data into 5 clusters using k-means
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 
# to change use .astype()

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 4, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1,
       1, 1, 4, 1, 1, 1, 4, 1, 4, 1, 1, 1, 1, 1, 2, 1, 1, 4, 1, 1, 1, 4,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 0, 1,
       4, 1, 1, 1, 1, 4, 3], dtype=int32)

In [100]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster_Labels_new', kmeans.labels_)

toronto_merged = Toronto_coords

In [101]:
toronto_merged.head()

,Cluster Labels,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,4,M3A,North York,Parkwoods,43.753259,-79.329656
1,4,M4A,North York,Victoria Village,43.725882,-79.315572
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,2,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [102]:
toronto_merged.rename(columns={'Neighborhood':'Neighbourhood'}, inplace=True)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head()

,Cluster Labels,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster_Labels_new,Cluster_Labels5,Cluster_Labels1,Cluster_Labels Venuues,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,4,M3A,North York,Parkwoods,43.753259,-79.329656,4.0,4.0,4.0,4.0,...,Park,Food & Drink Shop,BBQ Joint,Yoga Studio,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Donut Shop
1,4,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,1.0,1.0,1.0,...,Pizza Place,Intersection,Coffee Shop,Hockey Arena,Portuguese Restaurant,Yoga Studio,Dog Run,Dim Sum Restaurant,Diner,Discount Store
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1.0,1.0,1.0,1.0,...,Coffee Shop,Park,Pub,Bakery,Breakfast Spot,Restaurant,Café,Theater,Mexican Restaurant,Yoga Studio
3,3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,1.0,1.0,1.0,...,Furniture / Home Store,Clothing Store,Accessories Store,Athletics & Sports,Gift Shop,Event Space,Miscellaneous Shop,Coffee Shop,Boutique,Women's Store
4,2,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1.0,1.0,1.0,1.0,...,Coffee Shop,Yoga Studio,Diner,Nightclub,Mexican Restaurant,Juice Bar,Italian Restaurant,Indian Restaurant,Hobby Shop,Fried Chicken Joint


In [103]:
toronto_merged=toronto_merged.dropna()

In [104]:
toronto_merged['Cluster_Labels_new'] = toronto_merged.Cluster_Labels.astype(int)

In [105]:
toronto_merged.drop(['Cluster_Labels5','Cluster_Labels1'],axis=1)

,Cluster Labels,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster_Labels_new,Cluster_Labels Venuues,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,4,M3A,North York,Parkwoods,43.753259,-79.329656,4,4.0,4.0,Park,Food & Drink Shop,BBQ Joint,Yoga Studio,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Donut Shop
1,4,M4A,North York,Victoria Village,43.725882,-79.315572,1,1.0,1.0,Pizza Place,Intersection,Coffee Shop,Hockey Arena,Portuguese Restaurant,Yoga Studio,Dog Run,Dim Sum Restaurant,Diner,Discount Store
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,1.0,1.0,Coffee Shop,Park,Pub,Bakery,Breakfast Spot,Restaurant,Café,Theater,Mexican Restaurant,Yoga Studio
3,3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1,1.0,1.0,Furniture / Home Store,Clothing Store,Accessories Store,Athletics & Sports,Gift Shop,Event Space,Miscellaneous Shop,Coffee Shop,Boutique,Women's Store
4,2,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,1.0,1.0,Coffee Shop,Yoga Studio,Diner,Nightclub,Mexican Restaurant,Juice Bar,Italian Restaurant,Indian Restaurant,Hobby Shop,Fried Chicken Joint
6,0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,1,1.0,1.0,Fast Food Restaurant,Yoga Studio,Department Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant
7,4,M3B,North York,Don Mills,43.745906,-79.352188,1,1.0,1.0,Gym,Coffee Shop,Restaurant,Beer Store,Japanese Restaurant,Supermarket,Clothing Store,Chinese Restaurant,Caribbean Restaurant,Café
8,4,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,1,1.0,1.0,Pizza Place,Breakfast Spot,Fast Food Restaurant,Intersection,Athletics & Sports,Bus Line,Bank,Gastropub,Pharmacy,Gym / Fitness Center
9,2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,1.0,1.0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Ramen Restaurant,Thai Restaurant,Restaurant
10,3,M6B,North York,Glencairn,43.709577,-79.445073,1,1.0,1.0,Park,Pub,Sushi Restaurant,Japanese Restaurant,Pizza Place,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run


In [87]:
toronto_merged.head()

,Cluster Labels,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster_Labels Venuues,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,4,M3A,North York,Parkwoods,43.753259,-79.329656,4.0,4,Park,Food & Drink Shop,BBQ Joint,Yoga Studio,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Donut Shop
1,4,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,1,Pizza Place,Intersection,Coffee Shop,Hockey Arena,Portuguese Restaurant,Yoga Studio,Dog Run,Dim Sum Restaurant,Diner,Discount Store
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1.0,1,Coffee Shop,Park,Pub,Bakery,Breakfast Spot,Restaurant,Café,Theater,Mexican Restaurant,Yoga Studio
3,3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1.0,1,Furniture / Home Store,Clothing Store,Accessories Store,Athletics & Sports,Gift Shop,Event Space,Miscellaneous Shop,Coffee Shop,Boutique,Women's Store
4,2,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1.0,1,Coffee Shop,Yoga Studio,Diner,Nightclub,Mexican Restaurant,Juice Bar,Italian Restaurant,Indian Restaurant,Hobby Shop,Fried Chicken Joint


### Create a map to visualize clusters based on vanues frequency - column 'Cluster_Labels'

In [106]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster_Labels_new']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

Cluster 1

In [107]:
toronto_merged.loc[toronto_merged['Cluster_Labels_new'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Longitude,Cluster_Labels_new,Cluster_Labels5,Cluster_Labels1,Cluster_Labels Venuues,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,M9B,-79.554724,0,0.0,0.0,0.0,0.0,Home Service,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant,Deli / Bodega


Cluster 2

In [108]:
toronto_merged.loc[toronto_merged['Cluster_Labels_new'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Longitude,Cluster_Labels_new,Cluster_Labels5,Cluster_Labels1,Cluster_Labels Venuues,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M4A,-79.315572,1,1.0,1.0,1.0,1.0,Pizza Place,Intersection,Coffee Shop,Hockey Arena,Portuguese Restaurant,Yoga Studio,Dog Run,Dim Sum Restaurant,Diner,Discount Store
2,M5A,-79.360636,1,1.0,1.0,1.0,1.0,Coffee Shop,Park,Pub,Bakery,Breakfast Spot,Restaurant,Café,Theater,Mexican Restaurant,Yoga Studio
3,M6A,-79.464763,1,1.0,1.0,1.0,1.0,Furniture / Home Store,Clothing Store,Accessories Store,Athletics & Sports,Gift Shop,Event Space,Miscellaneous Shop,Coffee Shop,Boutique,Women's Store
4,M7A,-79.389494,1,1.0,1.0,1.0,1.0,Coffee Shop,Yoga Studio,Diner,Nightclub,Mexican Restaurant,Juice Bar,Italian Restaurant,Indian Restaurant,Hobby Shop,Fried Chicken Joint
6,M1B,-79.194353,1,1.0,1.0,1.0,1.0,Fast Food Restaurant,Yoga Studio,Department Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant
7,M3B,-79.352188,1,1.0,1.0,1.0,1.0,Gym,Coffee Shop,Restaurant,Beer Store,Japanese Restaurant,Supermarket,Clothing Store,Chinese Restaurant,Caribbean Restaurant,Café
8,M4B,-79.309937,1,1.0,1.0,1.0,1.0,Pizza Place,Breakfast Spot,Fast Food Restaurant,Intersection,Athletics & Sports,Bus Line,Bank,Gastropub,Pharmacy,Gym / Fitness Center
9,M5B,-79.378937,1,1.0,1.0,1.0,1.0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Ramen Restaurant,Thai Restaurant,Restaurant
10,M6B,-79.445073,1,1.0,1.0,1.0,1.0,Park,Pub,Sushi Restaurant,Japanese Restaurant,Pizza Place,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
12,M1C,-79.160497,1,1.0,1.0,1.0,1.0,Bar,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop


Cluster 3

In [109]:
toronto_merged.loc[toronto_merged['Cluster_Labels_new'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Longitude,Cluster_Labels_new,Cluster_Labels5,Cluster_Labels1,Cluster_Labels Venuues,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,M9M,-79.532242,2,2.0,2.0,2.0,2.0,Baseball Field,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
101,M8Y,-79.498509,2,2.0,2.0,2.0,2.0,Baseball Field,Yoga Studio,Dessert Shop,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop


Cluster 4

In [110]:
toronto_merged.loc[toronto_merged['Cluster_Labels_new'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Longitude,Cluster_Labels_new,Cluster_Labels5,Cluster_Labels1,Cluster_Labels Venuues,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,M2L,-79.374714,3,3.0,3.0,3.0,3.0,Cafeteria,Yoga Studio,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant


Cluster 5

In [111]:
toronto_merged.loc[toronto_merged['Cluster_Labels_new'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,PostalCode,Longitude,Cluster_Labels_new,Cluster_Labels5,Cluster_Labels1,Cluster_Labels Venuues,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,-79.329656,4,4.0,4.0,4.0,4.0,Park,Food & Drink Shop,BBQ Joint,Yoga Studio,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Donut Shop
21,M6E,-79.453512,4,4.0,4.0,4.0,4.0,Park,Women's Store,Market,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Drugstore
35,M4J,-79.338106,4,4.0,4.0,4.0,4.0,Park,Coffee Shop,Convenience Store,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
61,M4N,-79.388790,4,4.0,4.0,4.0,4.0,Park,Bus Line,Swim School,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant,Event Space
64,M9N,-79.518188,4,4.0,4.0,4.0,4.0,Park,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
66,M2P,-79.400049,4,4.0,4.0,4.0,4.0,Park,Bank,Bar,Convenience Store,Yoga Studio,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run
68,M5P,-79.411307,4,4.0,4.0,4.0,4.0,Park,Trail,Sushi Restaurant,Jewelry Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
83,M4T,-79.383160,4,4.0,4.0,4.0,4.0,Playground,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
85,M1V,-79.284577,4,4.0,4.0,4.0,4.0,Park,Playground,Yoga Studio,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Donut Shop
91,M4W,-79.377529,4,4.0,4.0,4.0,4.0,Park,Trail,Playground,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
